<a href="https://colab.research.google.com/github/TImrosalia/Minhas-Imersoes/blob/main/Sistema_de_busca_Embeddings_e_a_Gemini_API_Aula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando SDK

In [1]:
!pip install -U -q google.generativeai

In [36]:
#importação e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Escolhendo o modelo

In [37]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
 #Exemplo de emebedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

#iniciando com variável e metodo
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)


{'embedding': [0.046409428, -0.03581066, -0.020875515, 0.013134151, 0.06889433, -0.006942863, -0.025423253, -0.020147229, 0.0667757, 0.06341314, 0.014015882, 0.01526038, -0.049111225, -0.022847965, 0.0076156845, -0.02576318, 0.026596375, -0.010523227, -0.031838775, -0.0011655056, 0.004944847, 0.003429059, -0.032589234, -0.06923391, -0.018453777, 0.021771021, 0.0076866113, -0.03855544, -0.039048176, 0.02721454, -0.047767907, 0.053855136, -0.03261822, 0.009848433, -0.035794124, -0.037631024, -0.026956078, -0.046313386, -0.0011535396, -0.004269161, 0.00717925, -0.088856675, -0.0071202833, 0.02485196, -0.0057493746, -0.026181517, 0.047007352, 0.039841782, 0.012793134, -0.059709117, 0.029405339, 0.021644386, 0.064779684, -0.04527396, -0.0066189426, -0.0044017234, 0.022867665, -0.034612592, 0.023836834, -0.0021367038, -0.003599733, 0.023864374, -0.021040712, 0.052181933, 0.020639826, -0.056478173, -0.041059755, -0.0008751397, 0.012259135, 0.04740651, 0.011356011, 0.010642914, 0.06563923, -0.

Lista de documentos que serão usados para a busca

In [6]:
DOCUMENT1 = {
    "title": "Operação do sistema de controle climático",
    "content": "Seu Googlecar possui um sistema de controle de temperatura que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central. Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuí-lo. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite selecionar o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Legal: O carro soprará ar frio para dentro do carro. Calor: O carro soprará ar quente para dentro do carro. Descongelamento: O carro soprará ar quente no pára-brisa para descongelá-lo"}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Seu Googlecar possui uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle de temperatura. Para utilizar o display touchscreen, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone \"Navegação\" para obter rotas até seu destino ou tocar no ícone \"Música\" para reproduzir suas músicas favoritas."}
DOCUMENT3 = {
    "title": "Mudança de marchas",
    "content": "Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de câmbio para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverso: Esta posição é usada para fazer backup. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para avançar. Baixo: Esta posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

Para acessar data frames através do panda

In [11]:
df = pd.DataFrame(documents)
df

,title,content
0,Operação do sistema de controle climático,Seu Googlecar possui um sistema de controle de...
1,Touchscreen,Seu Googlecar possui uma grande tela sensível ...
2,Mudança de marchas,Seu Googlecar possui transmissão automática. P...


In [14]:
df = pd.DataFrame(documents)
df.columns = ["Titulos", "Conteudos"]
df

,Titulos,Conteudos
0,Operação do sistema de controle climático,Seu Googlecar possui um sistema de controle de...
1,Touchscreen,Seu Googlecar possui uma grande tela sensível ...
2,Mudança de marchas,Seu Googlecar possui transmissão automática. P...


Gerando função e salvando somente o texto numerico do embedding

In [15]:
model = "models/embedding-001"

In [25]:
#Confira os embeddings de cada um desses corpos de texto. Adicione essas informações ao DataFrame
def embed_fn(title, text):
#google libera aí
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Criar uma nova coluna

In [26]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulos"], row["Conteudos"]), axis=1)
df

,Titulos,Conteudos,Embeddings
0,Operação do sistema de controle climático,Seu Googlecar possui um sistema de controle de...,"[-0.016297633, -0.0338351, -0.02996529, 0.0485..."
1,Touchscreen,Seu Googlecar possui uma grande tela sensível ...,"[0.020514729, -0.039281264, -0.0052330005, -0...."
2,Mudança de marchas,Seu Googlecar possui transmissão automática. P...,"[-0.010746703, -0.025243483, -0.014119012, 0.0..."


Fazendo consultas, perguntas e respostas

In [29]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]
#A consulta é feita, o embedding é gerado
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
#O calculo de distância é feito, e o que estiver mais próximo será o indice do token
  indice = np.argmax(produtos_escalares)
#A partir do indice será dado o conteudo compatível para compreensão do usuário
  return df.iloc[indice]["Conteudos"]

In [30]:
consulta = "Como faço para trocar de marchas em um carro do google?"


trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de câmbio para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverso: Esta posição é usada para fazer backup. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para avançar. Baixo: Esta posição é usada para dirigir na neve ou em outras condições escorregadias.


Consulta dinâmica

In [31]:
#Quanto menor a temperatura, menor a alucinação
generation_config = {
    "temperature": 0.5,
    "candidate_count":1,
    }

In [32]:
#f' usado para juntar texto com variável
prompt = f'Reescreva esse texto de uma forma mais descontráida, sem adicionar informações que não façam parte do texto: {trecho}'

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia de Câmbio do Seu Googlecar:**

* **Estacionado:** Pra quando você quer dar um tempo e deixar o carro paradão.
* **Ré:** Quando você precisa dar uma voltinha pra trás.
* **Neutro:** Serve pra ficar parado no farol ou no engarrafamento, mas o carro não vai mexer a menos que você pise no acelerador.
* **Dirigir:** Pra ir pra frente, né?
* **Baixo:** Pra quando a coisa tá escorregadia, tipo na neve.


Biblioteca usada com embeddings com resultado de uma fonte real, com  informações verdadeiras e utilizando o gemini para reescrever esse conteúdo de uma forma mais criativa. Pode ser feito em json, em tópicos, em forma de poema, etc..